In [1]:
import requests
import json
import urllib3
import warnings
import time
# Suppress insecure HTTPS warnings for self-signed certs
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)
host = "re-cluster1.ps-redislabs.org"     # e.g. "re-cluster.ps-redislabs.org"
port = "9443"
username = "admin@rl.org"
password = "QFJ5GWf"
# Endpoint for the REST API to create a database
url_db = f"https://{host}:{port}/v1/bdbs"
payload_db = {
    "name": "v1-api-db",
    "type": "redis",                       # This defines it as a v1 database
    "memory_size": 2 * 1024 * 1024 * 1024,  # 1 GB memory size
}
# Create the database
response_db = requests.post(
    url_db,
    auth=(username, password),
    headers={
        "Content-Type": "application/json",
        "Accept": "application/json"
    },
    data=json.dumps(payload_db),
    verify=False  # Skip cert check (for self-signed certs/dev only)
)
print(f"Database Creation - {response_db.status_code} {response_db.reason}")
try:
    db_info = response_db.json()
    print(json.dumps(db_info, indent=4))
    db_uid = db_info.get("uid")  # Extracting the database UID
except Exception:
    print(response_db.text)
# Proceed to create users only if the database was created successfully
url_user = f"https://{host}:{port}/v1/users"
# User details to be created
users = [
    {
        "email": "john.doe@example.com",
        "name": "John Doe",
        "role": "db_viewer",
        "password": "asdf"  # Add a password
    },
    {
        "email": "mike.smith@example.com",
        "name": "Mike Smith",
        "role": "db_member",
        "password": "asdf"  # Add a password
    },
    {
        "email": "cary.johnson@example.com",
        "name": "Cary Johnson",
        "role": "admin",
        "password": "asdf"  # Add a password
    }
]
# Create users
for user in users:
    response_user = requests.post(
        url_user,
        auth=(username, password),
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json"
        },
        data=json.dumps(user),
        verify=False  # Skip cert check (for self-signed certs/dev only)
    )
    print(f"User Creation - {response_user.status_code} {response_user.reason}")
    try:
        print(json.dumps(response_user.json(), indent=4))
    except Exception:
        print(response_user.text)
# Fetch and display all users
response_fetch_users = requests.get(
    url_user,
    auth=(username, password),
    headers={
        "Accept": "application/json"
    },
    verify=False  # Skip cert check (for self-signed certs/dev only)
)
print("\nList of Users:")
if response_fetch_users.status_code == 200:
    users_list = response_fetch_users.json()
    for user in users_list:
        name = user.get("name")
        role = user.get("role")
        email = user.get("email")
        print(f"Name: {name}, Role: {role}, Email: {email}")
else:
    print(f"Failed to fetch users - {response_fetch_users.status_code}: {response_fetch_users.reason}")
    try:
        print(json.dumps(response_fetch_users.json(), indent=4))
    except Exception:
        print(response_fetch_users.text)
        
# Function to check if the DB is ready to be deleted
def is_db_deletable(db_uid):
    url_check = f"https://{host}:{port}/v1/bdbs/{db_uid}"
    resp = requests.get(
        url_check,
        auth=(username, password),
        headers={"Accept": "application/json"},
        verify=False
    )
    if resp.status_code == 200:
        status = resp.json().get("status")
        print(f"Database status: {status}")
        return status == "active"
    return False
# Retry loop to wait for DB to become deletable
max_attempts = 5
for attempt in range(max_attempts):
    if is_db_deletable(db_uid):
        # Now try to delete it
        url_delete_db = f"https://{host}:{port}/v1/bdbs/{db_uid}"
        response_delete_db = requests.delete(
            url_delete_db,
            auth=(username, password),
            headers={"Accept": "application/json"},
            verify=False
        )
        print(f"\nDatabase Deletion - {response_delete_db.status_code} {response_delete_db.reason}")
        if response_delete_db.status_code == 204:
            print("Database deleted successfully.")
        else:
            print("Failed to delete database.")
            try:
                print(json.dumps(response_delete_db.json(), indent=4))
            except Exception:
                print(response_delete_db.text)
        break
    else:
        print(f"Attempt {attempt+1}/{max_attempts}: Database not ready. Retrying in 5 seconds...\n")
        time.sleep(5)
else:
    print("Max retries exceeded. Database was not ready for deletion.")


Database Creation - 200 OK
{
    "acl": [],
    "action_uid": "fd5b75e0-9202-4285-bd5c-ef87522e379f",
    "aof_policy": "appendfsync-every-sec",
    "authentication_admin_pass": "sGDjgily8g3QjHgNfrprNVz7rNOueSvrz8KI1PEUvYG76jAY",
    "authentication_redis_pass": "",
    "authentication_sasl_pass": "",
    "authentication_sasl_uname": "",
    "authentication_ssl_client_certs": [],
    "authentication_ssl_crdt_certs": [],
    "authorized_subjects": [],
    "auto_upgrade": false,
    "background_op": [
        {
            "name": "SMCreateBDB",
            "progress": 0,
            "status": "pending"
        }
    ],
    "backup": false,
    "backup_failure_reason": "",
    "backup_history": 0,
    "backup_interval": 86400,
    "backup_progress": 0.0,
    "backup_status": "",
    "bigstore": false,
    "bigstore_ram_size": 0,
    "client_cert_subject_validation_type": "disabled",
    "conns": 5,
    "conns_type": "per-thread",
    "crdt": false,
    "crdt_causal_consistency": false,
 